In [33]:
import os
import torchvision.models as models 
import torch.nn as nn
import torch
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision
from torchinfo import summary
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as T
from tqdm.auto import tqdm
import torchmetrics
import cv2
from glob import glob
import requests
from io import StringIO
from urllib.parse import urlparse
os.environ["cuda_visible_devices"] = '0'
device0 = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size=1

In [34]:
url = "https://storage.googleapis.com/download/storage/v1/b/vitalring-seoul.appspot.com/o/ppgContinuous%2FsyFotyWOdfPVPfa8ZOkhJBodgpJ3%2FsyFotyWOdfPVPfa8ZOkhJBodgpJ3_1706757218000.csv?generation=1706757229270812&alt=media"
destination_path = "downloaded_file.csv"
response = requests.get(url)
response

<Response [200]>

In [39]:
if response.status_code == 200:
    # Assuming the CSV content is UTF-8 encoded
    csv_data = response.content.decode('utf-8')
    parsed_url = urlparse(url)
    filename = parsed_url.path.split('/')[-1]
    # Use pandas to read the CSV data into a DataFrame
    df = pd.read_csv(StringIO(csv_data))
    df.rename(columns={'PPG': 'AF', 'Type_1': 'PPG'}, inplace=True)
    # Now 'df' is a pandas DataFrame containing the CSV data
    print(df.head())
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

   AF       PPG
0   0  0.478063
1   0  0.458766
2   0  0.435854
3   0  0.419918
4   0  0.401380


In [52]:
import signal

class CustomDataset(Dataset):
    def __init__(self,data ,transform=None, target_transform=None):
        self.PPG_data=data
    def __len__(self):
        return len(self.PPG_data)

    def __getitem__(self, idx):

        signal_tensor=self.PPG_data[idx]
        
        return signal_tensor

df2=[]
test_data=torch.empty(len(df)//2500+1,1,3000)
for i in range(len(df)//2500):
    if len(df)//2500-1==i:
        signal_csv=df.loc[i*2500-499:(i+1)*2500]['PPG']
        test_data[i]=torch.tensor(signal_csv.to_numpy())
        signal_csv=df.loc[len(df)-3000:]['PPG']
        test_data[i+1]=torch.tensor(signal_csv.to_numpy())
    else:
        signal_csv=df.loc[i*2500:(i+1)*2500+499]['PPG']
        test_data[i]=torch.tensor(signal_csv.to_numpy())
        
test_dataset=CustomDataset(file_list,test_data,test_label)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

8076     0.742675
8077     0.738639
8078     0.733217
8079     0.726389
8080     0.718197
           ...   
11071    0.587435
11072    0.579644
11073    0.572211
11074    0.558657
11075    0.546675
Name: PPG, Length: 3000, dtype: float64

torch.Size([5, 1, 3000])